In [2]:
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
endpoint_name = "meta-textgeneration-llama-2-7b-2023-10-10-17-16-41-545"
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "*"

In [ ]:
!pip3 install new_dist/*.whl --upgrade --upgrade-strategy only-if-needed --force-reinstall;

In [5]:
from amazon_fmeval.data_loaders.data_config import DataConfig
from amazon_fmeval.model_runners.sm_jumpstart_model_runner import JumpStartModelRunner
from amazon_fmeval.eval_algo_mapping import get_eval_algorithm
from amazon_fmeval.constants import MIME_TYPE_JSONLINES
from amazon_fmeval.eval_algorithms.factual_knowledge import FactualKnowledge, FactualKnowledgeConfig

2023-10-11 10:37:34,933	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-11 10:37:35,080	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
config = DataConfig(
    dataset_name="tiny_dataset",
    dataset_uri="tiny_dataset.jsonl",
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="question",
    target_output_location="answer",
)

In [7]:
js_model_runner = JumpStartModelRunner(
    endpoint_name=endpoint_name,
    model_id=model_id,
    model_version=model_version,
    output='[0].generation',
    content_template='{"inputs":"$prompt"}',
    custom_attributes="accept_eula=true"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [8]:
eval_algo = FactualKnowledge(FactualKnowledgeConfig(target_output_delimiter="<OR>"))

In [9]:
eval_output = eval_algo.evaluate(model=js_model_runner, dataset_config=config, prompt_template="$feature")

2023-10-11 10:37:40,163	WARNING services.py:1832 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8326742016 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-10-11 10:37:40,303	INFO worker.py:1621 -- Started a local Ray instance.
2023-10-11 10:37:42,184	INFO read_api.py:374 -- To satisfy the requested parallelism of 32, each read task output will be split into 32 smaller blocks.
2023-10-11 10:37:43,256	INFO dataset.py:2180 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2023-10-11 10:37:43,259	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCusto

(_MapWorker pid=26910) sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
(_MapWorker pid=26910) sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


- Aggregate 1:   0%|          | 0/32 [00:00<?, ?it/s]

Shuffle Map 2:   0%|          | 0/32 [00:00<?, ?it/s]

Shuffle Reduce 3:   0%|          | 0/32 [00:00<?, ?it/s]

Running 0:   0%|          | 0/32 [00:00<?, ?it/s]

2023-10-11 10:37:57,939	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCustomJSON->SplitBlocks(32)] -> ActorPoolMapOperator[MapBatches(process_batch)->Map(ModelRunnerWrapper)] -> TaskPoolMapOperator[MapBatches(process_batch)] -> LimitOperator[limit=1]
2023-10-11 10:37:57,940	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-10-11 10:37:57,940	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`
2023-10-11 10:37:58,005	INFO actor_pool_map_operator.py:117 -- MapBatches(process_batch)->Map(ModelRunnerWrapper): Waiting for 8 pool actors to start...


(_MapWorker pid=27532) sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml [repeated 18x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_MapWorker pid=27532) sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml [repeated 18x across cluster]


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
print(f"Evaluation Output: {eval_output}")

Evaluation Output: [EvalOutput(eval_name='factual_knowledge', dataset_name='tiny_dataset', dataset_scores=[EvalScore(name='factual_knowledge', value=0.0625)], prompt_template='$feature', category_scores=None, output_path='/tmp/eval_results/')]
